In [1]:
# %load_ext autoreload
# %autoreload 2

import os

os.chdir("..")

import torch
import wandb
import torchvision
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from src.utils.download import download_models
from src.utils.uap_helper import generate_adversarial_images_from_model_dataset, get_datamodule
from src.utils.metrics import metrics, Metrics
from src.data.mri import MRIDataModule
from src.data.covidx import COVIDXDataModule
from src.utils.evaluation import WeightsandBiasEval
from src.models.imageclassifier import ImageClassifier

In [2]:
ENTITY = "24FS_I4DS27"
PROJECT = "baselines"
NUM_WORKERS = 0

transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((224, 224), antialias=True),
    ]
)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

In [3]:
# Download models if not present
models = download_models(ENTITY, PROJECT)

In [4]:
for model in tqdm(models, desc="Model", position=0, total=len(models)):
    modelname, dataset = model.split("-")
    print(f"\n---\nModel: {modelname} - Dataset: {dataset}")

    perturbations = generate_adversarial_images_from_model_dataset(
        modelname,
        dataset,
        transform,
        num_universal_pertubation_images=1,
        num_pertubation_images=100,
        desired_fooling_rate=1,
        norm_p=2,
        norm_alpha=0.00001,
        max_iter_image=20,
        seed=42,
        eps=1e-6,
        num_workers=NUM_WORKERS,
        device=device,
        verbose=True,
    )

    break

Model:   0%|          | 0/22 [00:00<?, ?it/s]


---
Model: efficientnet_v2_l - Dataset: covidx_data


Universal Pertubation:   0%|          | 0/1 [00:00<?, ?it/s]

Starting epoch 1...


Image:   0%|          | 0/100 [00:00<?, ?it/s]

...

Image:   0%|          | 0/100 [00:00<?, ?it/s]

Fooling rate: 0.94
Starting epoch 34...


Image:   0%|          | 0/100 [00:00<?, ?it/s]

Image 0...
Image fooled! Adding perturbation...
Image 1...
Image fooled! Adding perturbation...
Image 2...
Image fooled! Adding perturbation...
Image 3...
Loss: 2.717554807662964, Norm: 0.0, InvBCE: 2.6801018714904785
Loss: 2.407351493835449, Norm: 0.00038118421798571944, InvBCE: 2.3698573112487793
Loss: 2.134324550628662, Norm: 0.0007559842779301107, InvBCE: 2.0967862606048584


KeyboardInterrupt: 

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import torch

v = perturbations[0].detach()
get_datamodule(dataset, seed=42).setup()
image, _ = next(iter(get_datamodule(dataset).train_dataloader()))

# Prepare subplots in one row
fig, axs = plt.subplots(1, 4, figsize=(20, 5))  # Adjust figsize to ensure each subplot is visually proportional

# Original Image
axs[0].imshow(image.cpu().squeeze().permute(1, 2, 0).numpy().astype(int))
axs[0].set_title("Original Image", fontsize=16)
axs[0].axis("off")  # Hide axes for cleaner look

# Perturbed Image
x_adv = image + v.cpu()
x_adv = torch.clamp(x_adv, 0, 255)
axs[1].imshow(x_adv.cpu().squeeze().permute(1, 2, 0).numpy().astype(int))
axs[1].set_title("Perturbed Image", fontsize=16)
axs[1].axis("off")

# Perturbation Visualization
perturbation_visual = v.mean(dim=0).cpu().squeeze().numpy().astype(int)
vmax = abs(perturbation_visual).max()
im = axs[2].imshow(perturbation_visual, cmap="coolwarm", vmin=-vmax, vmax=vmax)
axs[2].set_title("Universal Perturbation", fontsize=16)
axs[2].axis("off")
cbar = plt.colorbar(im, ax=axs[2], fraction=0.046, pad=0.04)
cbar.set_label("Intensity", fontsize=12)

# Statistical Distribution of Perturbation
axs[3].hist(v.cpu().flatten(), bins=100, color="grey", alpha=0.7)
axs[3].set_aspect("auto", adjustable="box")
axs[3].set_title("Perturbation Distribution", fontsize=16)
axs[3].set_yticks([])
axs[3].axis("on")

plt.suptitle(
    f"Pixel Perturbation stats: min: {v.min().item():.2f}, max: {v.max().item():.2f}, σ: {v.std().item():.2f}, μ: {v.mean().item():.2f}",
    fontsize=12,
    y=0.05,  # Adjust for top layout
)

plt.tight_layout()
plt.show()

NameError: name 'perturbations' is not defined